# 🚀 Multi-Agent Systems & Workflow Patterns

**Welcome to the Kaggle 5-day Agents course!**

In the previous notebook, you built a **single agent** that could take action. Now, you'll learn how to scale up by building **agent teams**.

Just like a team of people, you can create specialized agents that collaborate to solve complex problems. This is called a **multi-agent system**, and it's one of the most powerful concepts in AI agent development.

In this notebook, you'll:

- ✅ Learn when to use multi-agent systems in [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- ✅ Build your first system using an LLM as a "manager"
- ✅ Learn three core workflow patterns (Sequential, Parallel, and Loop) to coordinate your agent teams


In [2]:
from dotenv import load_dotenv
import os

# loads variables from .env
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("Environment variable GOOGLE_API_KEY not found!")

print("Secret loaded successfully!")

Secret loaded successfully!


In [5]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅ Gemini API key setup complete.")

✅ Gemini API key setup complete.


In [7]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [9]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

---
## 🤔 Section 2: Why Multi-Agent Systems? + Your First Multi-Agent

**The Problem: The "Do-It-All" Agent**

Single agents can do a lot. But what happens when the task gets complex? A single "monolithic" agent that tries to do research, writing, editing, and fact-checking all at once becomes a problem. Its instruction prompt gets long and confusing. It's hard to debug (which part failed?), difficult to maintain, and often produces unreliable results.

**The Solution: A Team of Specialists**

Instead of one "do-it-all" agent, we can build a **multi-agent system**. This is a team of simple, specialized agents that collaborate, just like a real-world team. Each agent has one clear job (e.g., one agent *only* does research, another *only* writes). This makes them easier to build, easier to test, and much more powerful and reliable when working together.

To learn more, check out the documentation related to [LLM agents in ADK](https://google.github.io/adk-docs/agents/llm-agents/).

**Architecture: Single Agent vs Multi-Agent Team**

```mermaid
graph TD
    subgraph Single["❌ Monolithic Agent"]
        A["One Agent Does Everything"]
    end

    subgraph Multi["✅ Multi-Agent Team"]
        B["Root Coordinator"] -- > C["Research Specialist"]
        B -- > E["Summary Specialist"]

        C -- >|findings| F["Shared State"]
        E -- >|summary| F
    end

    style A fill:#ffcccc
    style B fill:#ccffcc
    style F fill:#ffffcc
```

<img width="800" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/multi-agent-team.png" alt="Multi-agent Team" />

### 2.1 Example: Research & Summarization System

Let's build a system with two specialized agents:

1. **Research Agent** - Searches for information using Google Search
2. **Summarizer Agent** - Creates concise summaries from research findings

In [30]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [32]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


Refer to the ADK documentation for more information on [guiding agents with clear and specific instructions](https://google.github.io/adk-docs/agents/llm-agents/).

Then we bring the agents together under a root agent, or coordinator:

In [36]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


Here we're using `AgentTool` to wrap the sub-agents to make them callable tools for the root agent. We'll explore `AgentTool` in-detail on Day 2.

Let's run the agent and ask it about a topic:

In [39]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


ResearchCoordinator > Quantum computing advancements are set to revolutionize AI by significantly enhancing its performance, data processing capabilities, and optimization potential. This synergy, known as Quantum AI, promises more efficient and sustainable AI solutions. Key implications include accelerated AI training, improved natural language processing, and the ability to tackle complex problems currently beyond classical computing's reach. Industries like healthcare, finance, and logistics are poised for transformation, with potential breakthroughs in personalized medicine, risk assessment, and supply chain optimization. While still developing, quantum computing's impact on AI is anticipated to be profound within the next decade.


You've just built your first multi-agent system! You used a single "coordinator" agent to manage the workflow, which is a powerful and flexible pattern.

‼️ However, **relying on an LLM's instructions to control the order can sometimes be unpredictable.** Next, we'll explore a different pattern that gives you guaranteed, step-by-step execution.

---

## 🚥 Section 3: Sequential Workflows - The Assembly Line

**The Problem: Unpredictable Order**

The previous multi-agent system worked, but it relied on a **detailed instruction prompt** to force the LLM to run steps in order. This can be unreliable. A complex LLM might decide to skip a step, run them in the wrong order, or get "stuck," making the process unpredictable.

**The Solution: A Fixed Pipeline**

When you need tasks to happen in a **guaranteed, specific order**, you can use a `SequentialAgent`. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.

**Use Sequential when:** Order matters, you need a linear pipeline, or each step builds on the previous one.

To learn more, check out the documentation related to [sequential agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/).

**Architecture: Blog Post Creation Pipeline**

```mermaid
graph LR
    A["User Input: Blog about AI"] -- > B["Outline Agent"]
    B -- >|blog_outline| C["Writer Agent"]
    C -- >|blog_draft| D["Editor Agent"]
    D -- >|final_blog| E["Output"]

    style B fill:#ffcccc
    style C fill:#ccffcc
    style D fill:#ccccff
```

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/sequential-agent.png" alt="Sequential Agent" />

### 3.1 Example: Blog Post Creation with Sequential Agents

Let's build a system with three specialized agents:

1. **Outline Agent** - Creates a blog outline for a given topic
2. **Writer Agent** - Writes a blog post
3. **Editor Agent** - Edits a blog post draft for clarity and structure

In [50]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [52]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",  # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [54]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


Then we bring the agents together under a sequential agent, which runs the agents in the order that they are listed:

In [57]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [59]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)


 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > Okay, I am OutlineAgent. Here is a blog post outline about the benefits of multi-agent systems for software developers.

## Blog Post Outline: Multi-Agent Systems for Software Developers

---

### **Headline:** Supercharge Your Software: How Multi-Agent Systems Are Revolutionizing Development

---

### **Introduction Hook:**

*   Imagine a team of tireless, specialized developers working around the clock on your project, each expertly handling a different facet of the task. This isn't science fiction; it's the power of multi-agent systems (MAS) and they're poised to transform how software is built.

---

### **Main Sections:**

**1. Enhanced Problem Solving and Complexity Management**

*   **Decomposition & Specialization:** MAS allows complex problems to be broken down into smaller, more manageable tasks, with individual agents speciali

In [79]:
from IPython.display import display, Image, Markdown

Markdown(response[0].content.parts[0].text)

Okay, I am OutlineAgent. Here is a blog post outline about the benefits of multi-agent systems for software developers.

## Blog Post Outline: Multi-Agent Systems for Software Developers

---

### **Headline:** Supercharge Your Software: How Multi-Agent Systems Are Revolutionizing Development

---

### **Introduction Hook:**

*   Imagine a team of tireless, specialized developers working around the clock on your project, each expertly handling a different facet of the task. This isn't science fiction; it's the power of multi-agent systems (MAS) and they're poised to transform how software is built.

---

### **Main Sections:**

**1. Enhanced Problem Solving and Complexity Management**

*   **Decomposition & Specialization:** MAS allows complex problems to be broken down into smaller, more manageable tasks, with individual agents specializing in specific areas (e.g., UI, database, security, testing).
*   **Parallel Processing & Speed:** Different agents can work on their assigned tasks concurrently, significantly accelerating development cycles and reducing time-to-market.
*   **Adaptability to Changing Requirements:** As requirements evolve, specific agents can be modified or new ones introduced without disrupting the entire system, offering greater flexibility.

**2. Improved Efficiency and Resource Utilization**

*   **Automation of Repetitive Tasks:** Agents can be programmed to handle mundane, repetitive development tasks like code generation, unit testing, or deployment, freeing up human developers for more creative and strategic work.
*   **Optimized Resource Allocation:** Agents can dynamically manage and allocate computational resources, ensuring optimal performance and cost-effectiveness for large-scale applications.
*   **Reduced Human Error:** By automating error-prone tasks and implementing intelligent validation, MAS can lead to more robust and reliable software with fewer bugs.

**3. Fostering Collaboration and Knowledge Sharing**

*   **Agent-to-Agent Communication:** MAS facilitates sophisticated communication protocols between agents, allowing them to share information, coordinate actions, and learn from each other's experiences.
*   **Knowledge Base Augmentation:** Agents can continuously learn and update shared knowledge bases, leading to a more intelligent and self-improving development environment.
*   **Simulating Complex Scenarios:** Developers can use MAS to simulate real-world scenarios and user interactions, gaining valuable insights before deployment and identifying potential issues early.

**4. Enabling Advanced Capabilities and Scalability**

*   **Decentralized Control & Robustness:** Systems composed of multiple agents are inherently more robust and fault-tolerant; if one agent fails, others can often compensate.
*   **Scalability for Demanding Applications:** MAS is ideal for building highly scalable applications that can handle massive amounts of data and concurrent users, such as in IoT, AI, or complex simulations.
*   **Emergent Behaviors and Innovation:** The interaction of multiple agents can lead to emergent behaviors that were not explicitly programmed, potentially uncovering novel solutions and driving innovation.

---

### **Concluding Thought:**

*   Multi-agent systems are no longer a niche concept; they represent a powerful paradigm shift for software developers. By embracing MAS, we can build more intelligent, efficient, and resilient software solutions, unlocking new levels of productivity and innovation in the digital age.

---

👏 Great job! You've now created a reliable "assembly line" using a sequential agent, where each step runs in a predictable order.

**This is perfect for tasks that build on each other, but it's slow if the tasks are independent.** Next, we'll look at how to run multiple agents at the same time to speed up your workflow.